In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os

In [2]:
file = "diy_mco_30-April'25.csv"
raw_data=pd.read_csv(file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)



In [3]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp
0,28421085,98a68ea0-6541-4ee6-a7d8-e37b5256ff30,28421085,156,LEAD_SUCCESSFULLY_CLOSED,2025-04-01 20:06:22.000,2025-04-01 20:12:16.000,ybl_rekyc_update,NaN,INDIVIDUAL,NaN,NaN,True,keoqnl49490768335764,2018-09-17 14:41:20.000,NaN,NaN,NaN,NaN
1,1163851909,b02b2dc8-9c5e-478b-9b92-509394dbc105,1163851909,156,LEAD_SUCCESSFULLY_CLOSED,2025-04-05 16:11:53.000,2025-04-06 14:38:24.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,LhpMxz10470043460451,2025-04-06 14:29:56.000,NaN,2025-04-06 14:28:36.000,2025-04-06 14:29:48.000,2025-04-06 14:29:48.000
2,1449517460,f1cadeaf-3fee-47f2-892a-7ea8892b477d,1449517460,156,LEAD_SUCCESSFULLY_CLOSED,2025-04-03 07:33:32.000,2025-04-03 10:15:06.000,business_name_update,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,wTAVEK30158869589427,2025-03-24 10:26:53.000,NaN,2025-04-03 07:37:28.000,2025-04-03 10:14:48.000,2025-04-03 10:14:48.000


In [4]:

raw_data.insert(5,'q',0)

In [5]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)

raw_data['q'].unique()

array(['Merchant Live', nan, 'Documents Rejected (Pending at Sales Team)',
       'Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections) ',
       'Lead Created/Posted (Pending at Sales team for  Document Upload)'],
      dtype=object)

In [6]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

Merchant Live                                                       122180
Documents Rejected (Pending at Sales Team)                           17111
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)          777
Lead Created/Posted (Pending at Sales team for  Document Upload)       119
Name: q, dtype: int64

In [7]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])


In [8]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [9]:
raw_data=raw_data.drop(columns=['remove_date'])
len(raw_data)


138126

In [10]:
raw_data['solution_type_level_2'].unique()

array(['ybl_rekyc_update', nan, 'business_name_update',
       'display_name_update', 'pan_addition', 'auth_signatory_update',
       'diy_upgrade', 'registered_address_update',
       'billing_address_update', 'gst_update', 'diy_edc'], dtype=object)

In [11]:
raw_data=raw_data[raw_data['solution_type_level_2'].isin(['diy_upgrade','']) | raw_data['solution_type_level_2'].isna()]
len(raw_data)

46896

In [12]:
raw_data['solution_type_level_2'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [13]:
print(len(raw_data))
raw_data.head(3)

46896


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp
1,1163851909,b02b2dc8-9c5e-478b-9b92-509394dbc105,1163851909,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-05 16:11:53.000,2025-04-06 14:38:24.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,LhpMxz10470043460451,2025-04-06 14:29:56.000,NaN,2025-04-06 14:28:36.000,2025-04-06 14:29:48.000,2025-04-06 14:29:48.000
7,1948087497,666fa9c5-5771-4f7f-8d45-781c5e6ff8c2,1948087497,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-05 05:22:45.000,2025-04-05 10:43:50.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,jdAvIM84108164505656,2025-04-05 10:35:09.000,NaN,2025-04-05 05:40:36.000,2025-04-05 10:35:01.000,2025-04-05 10:35:01.000
8,1336666364,920a909b-e8d4-4389-ac09-68ed13e18559,1336666364,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-20 13:14:15.000,2025-04-20 13:57:21.000,NaN,NaN,INDIVIDUAL,NaN,"{""PANEL_SUCCESS"":1}",True,oSAgEe97315463168259,2025-04-20 13:48:38.000,NaN,2025-04-20 13:43:23.000,2025-04-20 13:48:34.000,2025-04-20 13:48:34.000


In [14]:
raw_data.columns

Index(['cust_id', 'lead_id', 'fse_cust_id', 'workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at', 'solution_type_level_2',
       'solution_type_level_3', 'entity_type', 'products_context',
       'sbstage_count', 'name_match_status', 'mid', 'mid_creation_date',
       'agent_team', 'qc_pending_max_created', 'panelqc_max_created',
       'final_timestamp'],
      dtype='object')

In [15]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [16]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3','solution_type']]=raw_data[['cust_id', 'lead_id','entity_type','workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at','products_context','name_match_status','final_timestamp','panelqc_max_created','qc_pending_max_created','sbstage_count','fse_cust_id','agent_team','mid','mid_creation_date','solution_type_level_2','solution_type_level_3']]
       
len(data)

46896

In [17]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type
1,1163851909,b02b2dc8-9c5e-478b-9b92-509394dbc105,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-05 16:11:53.000,2025-04-06 14:38:24.000,NaN,NaN,...,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1163851909,NaN,LhpMxz10470043460451,2025-04-06 14:29:56.000,NaN,NaN
7,1948087497,666fa9c5-5771-4f7f-8d45-781c5e6ff8c2,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-05 05:22:45.000,2025-04-05 10:43:50.000,NaN,NaN,...,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1948087497,NaN,jdAvIM84108164505656,2025-04-05 10:35:09.000,NaN,NaN
8,1336666364,920a909b-e8d4-4389-ac09-68ed13e18559,INDIVIDUAL,156,LEAD_SUCCESSFULLY_CLOSED,Merchant Live,2025-04-20 13:14:15.000,2025-04-20 13:57:21.000,NaN,NaN,...,NaN,NaN,"{""PANEL_SUCCESS"":1}",NaN,1336666364,NaN,oSAgEe97315463168259,2025-04-20 13:48:38.000,NaN,NaN


In [18]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])


In [19]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [20]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-04-23 13:43:38.531995


In [21]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [22]:
data['Ageing']=data['Ageing'].fillna('')

In [23]:
data['Ageing'].value_counts()

               36085
Last 7 days     9206
Yesterday       1605
Name: Ageing, dtype: int64

In [24]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [25]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [26]:
data['Name_Match_Status'].value_counts()

Name Matched        40476
Name Not Matched     6420
Name: Name_Match_Status, dtype: int64

In [27]:
data['Product_Context'].value_counts()

Series([], Name: Product_Context, dtype: int64)

In [28]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [29]:
data['mid_created_date'].head(10)

1    2025-04-06 14:29:56
7    2025-04-05 10:35:09
8    2025-04-20 13:48:38
17   2025-04-15 12:55:56
21   2025-04-06 11:00:42
22                   NaT
23                   NaT
27   2017-12-27 16:58:55
38   2025-04-22 16:51:32
51   2025-04-24 09:56:07
Name: mid_created_date, dtype: datetime64[ns]

In [30]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [31]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-04-23 13:43:38.818265


In [32]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [33]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [34]:
current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [35]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [36]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    6713
6-7 Days             675
4-5 Days             636
2-3 Days             598
Less Than 1 Day      415
Name: Rejection Ageing, dtype: int64

In [37]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    6713
6-7 Days             675
4-5 Days             636
2-3 Days             598
Less Than 1 Day      415
Name: Rejection Ageing, dtype: int64

In [38]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [39]:
data['Entity_Type'].value_counts()

INDIVIDUAL            41736
PROPRIETORSHIP         5134
NON_PROPRIETORSHIP       26
Name: Entity_Type, dtype: int64

In [40]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [41]:
data['Lead Status'].value_counts()

Merchant Live                                                       37656
Documents Rejected (Pending at Sales Team)                           9037
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         200
Lead Created/Posted (Pending at Sales team for  Document Upload)        3
Name: Lead Status, dtype: int64

In [42]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

#null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


0


,Lead_ID,id


In [43]:
# raise Exception("Stop execution here")

print(len(data))

46896


In [44]:
#rejected_null_value=pd.read_csv('rejected_null_value.csv')

#rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

#print(rejected_null_value.columns)

In [45]:
#print(rejected_null_value.head(3))
print(len(data))

46896


In [46]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [47]:
#data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

#data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

#data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')
0
46896


In [48]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [49]:
# type(data['Final TimeStamp'][0])
#print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

46896


In [50]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [51]:
data['Last QC Pending Timestamp'].head(10)

1    2025-04-06 14:28:36
7    2025-04-05 05:40:36
8    2025-04-20 13:43:23
17   2025-04-15 12:54:40
21   2025-04-06 01:53:44
22   2025-04-12 17:06:16
23   2025-04-22 21:41:31
27   2025-04-25 11:02:03
38   2025-04-22 16:49:34
51   2025-04-24 09:53:58
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [52]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
#start_time = time(8, 30, 0)
#end_time = time(21, 30, 0)

start_time = time(8, 00, 0)
end_time = time(22, 00, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

1      Intime
7     Offtime
8      Intime
17     Intime
21    Offtime
22     Intime
23     Intime
27     Intime
38     Intime
51     Intime
Name: Working TimeSlab, dtype: object

In [53]:
data['Working TimeSlab'].value_counts()

Intime     39764
Offtime     7132
Name: Working TimeSlab, dtype: int64

In [54]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [55]:
#out_time_start = pd.to_datetime("21:30").time()
#out_time_end = pd.to_datetime("23:59:59").time()
#morning_time_end = pd.to_datetime("08:30").time()

out_time_start = pd.to_datetime("22:00").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:00").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [56]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=00, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [57]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [58]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [59]:
data['TAT'].head(10)

1    0 days 00:01:12
7    0 days 02:35:01
8    0 days 00:05:11
17   0 days 00:00:57
21   0 days 03:00:35
22   0 days 00:01:05
23   0 days 00:00:48
27   0 days 00:05:27
38   0 days 00:01:47
51   0 days 00:02:02
Name: TAT, dtype: timedelta64[ns]

In [60]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [61]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

       Final TimeStamp Last QC Pending Timestamp             TAT
1  2025-04-06 14:29:48       2025-04-06 14:28:36 0 days 00:01:12
7  2025-04-05 10:35:01       2025-04-05 05:40:36 0 days 02:35:01
8  2025-04-20 13:48:34       2025-04-20 13:43:23 0 days 00:05:11
17 2025-04-15 12:55:37       2025-04-15 12:54:40 0 days 00:00:57
21 2025-04-06 11:00:35       2025-04-06 01:53:44 0 days 03:00:35
22 2025-04-12 17:07:21       2025-04-12 17:06:16 0 days 00:01:05
23 2025-04-22 21:42:19       2025-04-22 21:41:31 0 days 00:00:48
27 2025-04-25 11:07:30       2025-04-25 11:02:03 0 days 00:05:27
38 2025-04-22 16:51:21       2025-04-22 16:49:34 0 days 00:01:47
51 2025-04-24 09:56:00       2025-04-24 09:53:58 0 days 00:02:02
46896


In [62]:
#len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())]

,Merchant Cust ID,Lead_ID,Entity_Type,Node_id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3,solution_type


In [63]:
data = data[~(((data['Lead Status'] == 'Merchant Live') | (data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)')) & (data['TAT'].isnull()))]


len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [64]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [65]:
data['Merchant QC TAT slab'].value_counts()

Leads QCed in less than 02 min          14824
leads QCed in more than 01 hr           14421
Leads QCed in between 02 - 05 min        8440
Leads QCed in between 05 - 10 min        3582
Leads QCed in between 30 min - 01 hr     2492
Leads QCed in between 15 - 30 min        1800
Leads QCed in between 10 - 15 min        1134
Name: Merchant QC TAT slab, dtype: int64

In [66]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node_id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3', 'solution_type'],
      dtype='object')

In [67]:
data['solution_type_level_3'].unique()

array([nan, 'diy_upgrade'], dtype=object)

In [68]:
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (data['solution_type_level_3'].isna()),'solution_type']='DIY_MCO'
data.loc[(data['Entity_Type']=='INDIVIDUAL') & (~data['solution_type_level_3'].isna()),'solution_type']='DIY_ReKYC'
data.loc[~(data['Entity_Type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [69]:
data['solution_type'].unique()

array(['DIY_MCO', 'DIY_Upgrade', 'DIY_ReKYC'], dtype=object)

In [70]:
data.to_csv("diy_mco_data_29th April'25.csv", index=False)

In [71]:
hzdjz

NameError: name 'hzdjz' is not defined

In [ ]:
#data.to_csv("all_flow_data_28th Oct'24.csv", index=False)

In [ ]:
sas

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
